## Globals

In [1]:
from __future__ import print_function

import sys
import os
import re
import glob
import subprocess
import shutil
import json


root_dir = '/project_cephfs/3022017.06/HCP_Baby'
cluster_log_dir = os.path.join(root_dir,'logs')
#aroma_path = '/home/preclineu/andmar/sfw/ICA-AROMA/ICA_AROMA.py'  

cmd_qsub_base = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '32gb',
                 '-logfiledir', cluster_log_dir
                ]

#abcd_dict = os.path.join(root_dir,'downloads','abcd_modalities.json')

# read the dictionary
#with open(abcd_dict,'r') as f:
#   modalities = json.load(f) 
    
# get a list of subjects to process
sub_dirs = sorted(glob.glob(os.path.join(root_dir,'freesurfer','MN*')))

#sub_dirs = sub_dirs[0:15]
#print('\n'.join(sub_dirs))

#print('Dictionary read from:', abcd_dict)
print('Found', len(sub_dirs), 'subjects to process')

Found 595 subjects to process


In [32]:
sub_dirs

['/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCO601017_v05-2-43mo-20191214',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP000080_v05-1-43mo-20191118',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP000178_v01-2-15mo-20190221',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP000195_v03-1-17mo-20191117',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP102606_v03-1-16mo-20200110',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP105040_v02-1-9mo-20170529',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP105040_v04-1-15mo-20171203',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP105040_v05-1-24mo-20180916',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP107842_v01-1-19mo-20171122',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP107842_v02-2-25mo-20180626',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP115072_v01-1-14mo-20170608',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP116056_v01-1-3mo-20161220',
 '/pro

## run fsl_anat

In [17]:
verbose = False

sub_batch = sub_dirs[:10]#[10000:]

submitted = 0
already_processed = 0
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0]
    if verbose:
        print('processing subject',subid)

    job_name = 'ABCD_fsl_anat_' + subid
    
    sessions = glob.glob(os.path.join(s, 'ses-*'))
    for sess in sessions:
        sdir = os.path.join(sess, 'anat')
        if not os.path.exists(sdir):
            continue
        
        # get all subdirectories 
        t1_all = [os.path.join(sdir, o) for o in os.listdir(sdir) if os.path.isdir(os.path.join(sdir, o))]

        for t1_dir in t1_all:
            if os.path.exists(os.path.join(t1_dir,'T1.anat','T1_biascorr_brain.nii.gz')):
                if verbose: 
                    print(' >', subid, ' anatomical data already processed. Doing nothing')
                already_processed += 1
                continue
    
            cmd_fsl_anat = ['fsl_anat -i ', os.path.join(t1_dir,'T1.nii.gz')]
            cmd_str = '"%s"' % str(' '.join(cmd_fsl_anat)) 
            cmd_qsub = cmd_qsub_base + ['-name', job_name,'-command', cmd_str]

            #subprocess.Popen(' '.join(cmd_qsub), shell=True)
            
            submitted += 1
    
print('number of jobs submitted =', submitted)
print('number of T1s already processed =', already_processed)

number of jobs submitted = 4
number of T1s already processed = 24


## Select subjects (e.g. only those having 4 year follow-up)

In [40]:
#target_ses = 'ses-4YearFollowUpYArm1'
#target_ses = 'ses-2YearFollowUpYArm1'
#target_ses = '_v01'

sub_list = []
sub_dirs_target = []
for s in sub_dirs:
    subid = os.path.splitext(os.path.basename(s))[0] 
    path_dir = os.path.dirname(s)+ "/" + subid.split("_")[0]
    
    #sessions = list(map(os.path.basename, ses_dirs))

    sub_list.append(subid)
    sub_dirs_target.append(path_dir)

print(len(sub_list), 'subjects found')

sub_list= list(set(sub_list))

495 subjects found


In [41]:
sub_dirs_target

['/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCO601017',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP000080',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP000178',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP000195',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP102606',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP105040',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP105040',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP105040',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP107842',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP107842',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP115072',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP116056',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP116056',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP116056',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP116056',
 '/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP1

## Run Freesurfer (cross-sectional)

In [ ]:
num_processed = 0
num_failed = 0
num_submitted = 0
sub_no_t1 = []

verbose = False

default_t1_type = 'ABCD-T1-NORM'
log_prefix = 'ABCD_fs_'

#sub_batch = sub_dirs[100:]
sub_batch = sub_dirs_target

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_batch), 'subjects ...')
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0] 
    
    sessions = glob.glob(os.path.join(s, 'ses-*'))
    for sess in sessions:
        sdir = os.path.join(sess, 'anat')
        sesid = os.path.basename(sess)
        full_subid = '_'.join((subid, sesid))
        
        if not os.path.exists(sdir):
            continue
        
        # get all subdirectories 
        t1_all = [os.path.join(sdir, o) for o in os.listdir(sdir) if os.path.isdir(os.path.join(sdir, o))]
        
        if len(t1_all) > 1:
            if verbose:
                print(' > Multiple t1s found for', full_subid, '(',len(t1_all), ') using default')
            
            # sub-select only ABCD-T1-NORM
            t1_all = [k for k in t1_all if default_t1_type in k]
            if len(t1_all) > 1:
                if verbose:
                    print(' > Failed.')
                continue;                        
        
        for t1_dir in t1_all:
            t1_file = os.path.join(t1_dir,'T1.nii.gz')            
            out_dir = os.path.join(root_dir, 'freesurfer', full_subid)
            
            if os.path.exists(out_dir):
                if os.path.exists(os.path.join(out_dir, 'stats', 'rh.w-g.pct.stats')):
                    # analysis is complete
                    if verbose:
                        print(' > Freesurfer already run for', full_subid,'doing nothing')
                    num_processed += 1
                    continue
                else:
                    # analysis did not complete
                    num_failed +=1
                    if verbose:
                        print(' > Freesurfer subject',full_subid,'found but failed. Continuing analysis')
                    cmd_fs = ['recon-all -s', full_subid, '-all','-qcache']
            else:
                # no analysis has been run
                if not os.path.exists(t1_file):
                    if verbose: 
                        print(' > No T1 found for sub', full_subid, '. Doing nothing')
                    sub_no_t1.append(subid)
                    continue
        
                cmd_fs = ['recon-all -i ', t1_file, '-s', full_subid, '-all','-qcache']
            
            #print(' '.join(cmd_fs))

            cmd_env = [fs_env + ' ; ',
                       'export SUBJECTS_DIR=' + os.path.join(root_dir,'freesurfer ; ')]
            cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
            cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(full_subid),'-command', cmd_str]
    
            num_submitted += 1
            #print(' '.join(cmd_qsub))
            subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)
print('number without t1 =',len(sub_no_t1))

## Create Freesurfer longitudinal templates

Note: this needs to be run more carefully than the cross sectional approach above because it just clobbers the template directory. 

In [42]:
sub_batch_list = []
for s in sub_dirs:
    #subid = os.path.splitext(os.path.basename(s))[0]
    id = s.split('_')[0] + '_' + s.split('_')[1]
    sub_batch_list.append(id)


sub_batch_list = list(set(sub_batch_list))
print(sub_batch_list)

['/project_cephfs/3022017.06/PING/freesurfer/P1495', '/project_cephfs/3022017.06/PING/freesurfer/P0733', '/project_cephfs/3022017.06/PING/freesurfer/P0485', '/project_cephfs/3022017.06/PING/freesurfer/P0442', '/project_cephfs/3022017.06/PING/freesurfer/P1386', '/project_cephfs/3022017.06/PING/freesurfer/P0564', '/project_cephfs/3022017.06/PING/freesurfer/P0862', '/project_cephfs/3022017.06/PING/freesurfer/P1136', '/project_cephfs/3022017.06/PING/freesurfer/P0289', '/project_cephfs/3022017.06/PING/freesurfer/P0383', '/project_cephfs/3022017.06/PING/freesurfer/P0964', '/project_cephfs/3022017.06/PING/freesurfer/P0053', '/project_cephfs/3022017.06/PING/freesurfer/P0158', '/project_cephfs/3022017.06/PING/freesurfer/P0360', '/project_cephfs/3022017.06/PING/freesurfer/P0530', '/project_cephfs/3022017.06/PING/freesurfer/P1330', '/project_cephfs/3022017.06/PING/freesurfer/P0911', '/project_cephfs/3022017.06/PING/freesurfer/P0035', '/project_cephfs/3022017.06/PING/freesurfer/P0514', '/project_c

In [45]:
num_processed = 0
num_failed = 0
num_submitted = 0

verbose = False

log_prefix = 'HCP_Baby'

#sub_batch = sub_dirs[10:]
sub_batch = sub_dirs_target

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_batch), 'subjects ...')
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0] 
    
    cmd_fs = ['recon-all -base ', subid + '_long',]
    
    sessions = sorted(glob.glob(s + '_*'))
    
    # make sure the baseline session is first 
    for i in range(len(sessions)):
        if '_v01' in sessions[i]:
            bl = sessions[i]
            del sessions[i]
            sessions.insert(0, bl)
            continue
    
    good_sessions = 0 
    for sess in sessions:
        sesid = os.path.basename(sess)
        full_subid = sesid 
        fs_sess_dir = os.path.join(root_dir, 'freesurfer', full_subid)
        
        fs_sessions = []
        #if os.path.exists(os.path.join(fs_sess_dir, 'stats', 'rh.w-g.pct.stats')):
        if os.path.exists(os.path.join(fs_sess_dir, 'stats', 'rh.BA_exvivo.thresh.stats')):
            fs_sessions.append(full_subid)
        else:
            print(' > Freesurfer did not complete for', full_subid)
            continue
        
        cmd_fs += ['-tp', full_subid]
        good_sessions += 1
        
    if good_sessions < 2:
        print(' > Only one timepoint for', full_subid, 'aborting.')
        continue
        
    cmd_fs += ['-all', '-qcache']
        
    print(' '.join(cmd_fs))

    cmd_env = [fs_env + ' ; ',
               'export SUBJECTS_DIR=' + os.path.join(root_dir,'freesurfer ; ')]
    cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
    cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(subid + '_long'),'-command', cmd_str]
    num_submitted += 1
    #print(' '.join(cmd_qsub))
    
    subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)

Processing 495 subjects ...
 > Only one timepoint for MNBCO601017_v05-2-43mo-20191214 aborting.
 > Only one timepoint for MNBCP000080_v05-1-43mo-20191118 aborting.
 > Only one timepoint for MNBCP000178_v01-2-15mo-20190221 aborting.
 > Only one timepoint for MNBCP000195_v03-1-17mo-20191117 aborting.
 > Only one timepoint for MNBCP102606_v03-1-16mo-20200110 aborting.
recon-all -base  MNBCP105040_long -tp MNBCP105040_v02-1-9mo-20170529 -tp MNBCP105040_v04-1-15mo-20171203 -tp MNBCP105040_v05-1-24mo-20180916 -all -qcache
recon-all -base  MNBCP105040_long -tp MNBCP105040_v02-1-9mo-20170529 -tp MNBCP105040_v04-1-15mo-20171203 -tp MNBCP105040_v05-1-24mo-20180916 -all -qcache
recon-all -base  MNBCP105040_long -tp MNBCP105040_v02-1-9mo-20170529 -tp MNBCP105040_v04-1-15mo-20171203 -tp MNBCP105040_v05-1-24mo-20180916 -all -qcache
recon-all -base  MNBCP107842_long -tp MNBCP107842_v01-1-19mo-20171122 -tp MNBCP107842_v02-2-25mo-20180626 -all -qcache
recon-all -base  MNBCP107842_long -tp MNBCP107842_v

In [30]:
sessions

[]

## Select subjects

In [2]:
fs_dir = os.path.join(root_dir,'freesurfer')
sub_dirs = glob.glob(os.path.join(fs_dir, '*_long'))

sub_list_target = []
for s in sub_dirs:
    subid = os.path.splitext(os.path.basename(s))[0].split('_')[0]
    
    sub_list_target.append(subid)
        
print(len(sub_list_target), 'subjects found')
sub_list_target[:10]

100 subjects found


['MNBCP198549',
 'MNBCP439999',
 'MNBCP518969',
 'MNBCP477275',
 'MNBCP755330',
 'MNBCP229768',
 'MNBCP839327',
 'MNBCP261266',
 'MNBCP107842',
 'MNBCP850013']

In [4]:
s

'/project_cephfs/3022017.06/HCP_Baby/freesurfer/MNBCP939785_long'

## Run longitudinal freesurfer

In [7]:
num_processed = 0
num_failed = 0
num_submitted = 0
sub_no_t1 = []

verbose = True

log_prefix = 'HCP_Babies_longfs2_'

sub_id_batch = sub_list_target

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_id_batch), 'subjects ...')
for subid in sub_id_batch:
        
    sessions = glob.glob(os.path.join(fs_dir, f'{subid}_*'))
    sessions = [item for item in sessions if 'long' not in item]
    for full_subid in sessions:
        sesid = os.path.basename(full_subid)
        
        #if not os.path.exists(os.path.join(fs_dir, full_subid)):
        #    continue
                  
        out_dir = os.path.join(root_dir, 'freesurfer', full_subid)
            
        #if os.path.exists(out_dir):
        #    if os.path.exists(os.path.join(out_dir, 'stats', 'rh.w-g.pct.stats')):
        #        # analysis is complete
        #        if verbose:
        #            print(' > Freesurfer already run for', full_subid,'doing nothing')
        #        num_processed += 1
        #        continue
        #    else:
        #         # analysis did not complete
        #        num_failed +=1
        #        if verbose:
        #            print(' > Freesurfer subject',full_subid,'found but failed. Continuing analysis')
        #        cmd_fs = ['recon-all -long ', full_subid, f'{sub_id}_long', '-all','-qcache']
        #else:
            # no analysis has been run
            #if not os.path.exists(t1_file):
            #    if verbose: 
            #        print(' > No T1 found for sub', full_subid, '. Doing nothing')
            #    sub_no_t1.append(subid)
            #    continue
        
        cmd_fs = ['recon-all -long ', full_subid, f'{subid}_long', '-all','-qcache']
            
        #print(' '.join(cmd_fs))

        cmd_env = [fs_env + ' ; ',
                    'export SUBJECTS_DIR=' + fs_dir + ' ; ']
        cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
        cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(full_subid),'-command', cmd_str]
    
        num_submitted += 1
        #print(' '.join(cmd_qsub))
        subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)
print('number without t1 =',len(sub_no_t1))

Processing 100 subjects ...
52838122.dccn-l029.dccn.nl
52838123.dccn-l029.dccn.nl
52838124.dccn-l029.dccn.nl
52838125.dccn-l029.dccn.nl
52838126.dccn-l029.dccn.nl
52838127.dccn-l029.dccn.nl
52838128.dccn-l029.dccn.nl
52838129.dccn-l029.dccn.nl
52838130.dccn-l029.dccn.nl
52838131.dccn-l029.dccn.nl
52838132.dccn-l029.dccn.nl
52838133.dccn-l029.dccn.nl
52838134.dccn-l029.dccn.nl
52838135.dccn-l029.dccn.nl
52838136.dccn-l029.dccn.nl
52838137.dccn-l029.dccn.nl
52838138.dccn-l029.dccn.nl
52838139.dccn-l029.dccn.nl
52838140.dccn-l029.dccn.nl
52838141.dccn-l029.dccn.nl
52838142.dccn-l029.dccn.nl
52838143.dccn-l029.dccn.nl
52838144.dccn-l029.dccn.nl
52838145.dccn-l029.dccn.nl
52838146.dccn-l029.dccn.nl
52838147.dccn-l029.dccn.nl
52838148.dccn-l029.dccn.nl
52838149.dccn-l029.dccn.nl
52838150.dccn-l029.dccn.nl
52838151.dccn-l029.dccn.nl
52838152.dccn-l029.dccn.nl
52838153.dccn-l029.dccn.nl
52838154.dccn-l029.dccn.nl
52838155.dccn-l029.dccn.nl
52838156.dccn-l029.dccn.nl
52838157.dccn-l029.dccn.nl
